In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import BloomForCausalLM
from transformers import BloomTokenizerFast

In [ ]:
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [ ]:
bloom_560m=BloomForCausalLM.from_pretrained("bigscience/bloom-560m")


In [ ]:
tokenizer=BloomTokenizerFast.from_pretrained("bigscience/bloom-560m")

In [ ]:
prompt = "what is a data scientist is"
result_lenth = 128

input = tokenizer(prompt, return_tensors="pt")

In [ ]:
result1=(tokenizer.decode(bloom_560m.generate(input["input_ids"],
                                              max_length=result_lenth,
                                              do_sample=True,
                                              top_k=50,
                                              top_p=0.9
                                              )[0]))
print("Q:",prompt)
print("result:",result1[len(prompt):])

Q: what is a data scientist is
result:  all about? What is a data scientist’s core skills? How does a data scientist interact with other disciplines in a way that is relevant to their specialties? Here are some examples:
The data scientist is a software engineer who has knowledge of databases and the use of tools to extract, analyze and visualize data. The data scientist is responsible for analyzing and transforming data.
The data scientist also functions as a researcher with the objective of collecting and analyzing data to achieve knowledge.
Data is the material that is a result of mathematical logic that has been coded into a system. Data is the material that is the


In [ ]:
!pip install datasets
!pip install transformers
!pip install accelerate==0.20.3


In [ ]:
from datasets import load_dataset
CSqa = load_dataset("commonsense_qa")

In [ ]:
import pandas as pd

common=pd.DataFrame(CSqa['validation'])

In [ ]:
def format_choices(choices):
    result = "\n".join(f"{label} {text}" for label, text in zip(choices['label'], choices['text']))
    return result
common['formatted_choices'] = common['choices'].apply(format_choices)

In [ ]:
def buildprompt(data):
    prompt = {}
    prompt['text'] = "answer the following question by specifying the answer key: \n"+ data['question'] +"\n" + data['formatted_choices']+ "\nAnswer: " + data['answerKey']
    return prompt

common['prompt'] = common.apply(buildprompt, axis=1)

In [ ]:
import json
result = common['prompt'].to_list()
with open('prompts.json', 'w') as outfile:
    json.dump(result, outfile, ensure_ascii=False)

In [ ]:
dataset = load_dataset("json", data_files="prompts.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
import torch

checkpoint = "bigscience/bloomz-560m"

tokenizer = BloomTokenizerFast.from_pretrained(checkpoint)
model = BloomForCausalLM.from_pretrained(checkpoint).to("cuda")

In [ ]:
def prepare_train_data(data):
    # prompt + completion
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(
    text_input,
    return_tensors='pt',
    padding='max_length',  # Pad to the maximum length
    max_length=93  # Set the maximum length
)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

train_dataset = dataset['train'].map(prepare_train_data,
                                     batched=True,
                                     remove_columns=["text"])

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [ ]:
training_arguments = TrainingArguments(
    'FT-bloom-560m',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True
)

In [ ]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,2.056300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=608, training_loss=1.768100198946501, metrics={'train_runtime': 628.1126, 'train_samples_per_second': 7.776, 'train_steps_per_second': 0.968, 'total_flos': 820006549364736.0, 'train_loss': 1.768100198946501, 'epoch': 3.98})

In [ ]:
trainer.save_model()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = BloomForCausalLM.from_pretrained("/content/FT-bloom-560m").to('cuda')

In [ ]:
import pandas as pd

In [ ]:
commontest=pd.read_csv('/content/Commontest-processed.csv')

In [ ]:
commontest

,Unnamed: 0,prompt,answerKey,BloomZ-560m,BloomZ-3b
0,0,answer the following question by specifying th...,A,C,D
1,1,answer the following question by specifying th...,B,A,B
2,2,answer the following question by specifying th...,B,M,A
3,3,answer the following question by specifying th...,D,M,D
4,4,answer the following question by specifying th...,C,C,D
...,...,...,...,...,...
120,120,answer the following question by specifying th...,E,E,B
121,121,answer the following question by specifying th...,A,B,A
122,122,answer the following question by specifying th...,D,D,D
123,123,answer the following question by specifying th...,C,A,D


In [ ]:
results = []
for i, P in enumerate(commontest["prompt"]):
    prompt = P
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to('cuda:0') for k, v in inputs.items()}  # Move inputs to cuda:0

    # Ensure the model is on the same device (cuda:0)
    model.to('cuda:0')

    result = (tokenizer.decode(model.generate(inputs["input_ids"],
                       max_length=len(prompt) + 1,
                       do_sample=True,
                       top_k=50,
                       top_p=0.9
                      )[0]))
    results.append(result[len(prompt)+1])

In [ ]:
len(results)

125

In [ ]:
commontest['BloomZ-ft']=results

In [ ]:
commontest

,Unnamed: 0,prompt,answerKey,BloomZ-560m,BloomZ-3b,BloomZ-ft
0,0,answer the following question by specifying th...,A,C,D,D
1,1,answer the following question by specifying th...,B,A,B,A
2,2,answer the following question by specifying th...,B,M,A,B
3,3,answer the following question by specifying th...,D,M,D,A
4,4,answer the following question by specifying th...,C,C,D,D
...,...,...,...,...,...,...
120,120,answer the following question by specifying th...,E,E,B,A
121,121,answer the following question by specifying th...,A,B,A,B
122,122,answer the following question by specifying th...,D,D,D,A
123,123,answer the following question by specifying th...,C,A,D,̉


In [ ]:
def replace_values(value):
    if value not in ['A', 'B', 'C', 'D', 'E']:
        return 'M'
    else:
        return value